<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/phase3/phase3_aboumahdi_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/Thesis/phase-3/tokenized_phase3.json ./train.json

In [3]:
!pip install transformers

     |████████████████████████████████| 450kB 2.9MB/s 
     |████████████████████████████████| 1.0MB 9.4MB/s 
     |████████████████████████████████| 870kB 20.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=880f4e7395b060d2abda539501176d7a425f1fdf7bd4fe85c708a869038e4e29
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [4]:
import torch
from transformers import DistilBertTokenizer, GPT2Tokenizer
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilgpt_tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

In [0]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import json
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, json_file):
        """
        Args:
            json_file (string): Path to the json file with annotations.
        """
        self.dialogues = json.load(open(json_file))
        self.root_dir = json_file

        self.dialogues = sorted(self.dialogues, key=lambda x: len(x['history']))

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
      
        
        history_lst = self.dialogues[idx]['history']
        knowledge_lst = self.dialogues[idx]['knowledge']
        response_lst = self.dialogues[idx]['response']


        if(len(history_lst)>100):
          history_lst = history_lst[-100:]

        if(len(knowledge_lst)>50):
          knowledge_lst = knowledge_lst[:50]
        
        if(len(response_lst)>50):
          response_lst = response_lst[:50]
        

        history = torch.LongTensor(history_lst)
        knowledge = torch.LongTensor(knowledge_lst)
        response = torch.LongTensor(response_lst)

        sample = {'history': history, 'knowledge': knowledge, 'response': response}

        return sample

In [0]:
dataset = MyDataset('train.json')

In [20]:
print('len dataset is: {}'.format(len(dataset)))

len dataset is: 74092


In [0]:
def my_collate_fn(batch):

  len_batch = len(batch)

  
  max_len_history = max([len(data['history']) for data in batch])
  max_len_knowledge_sample = max([len(data['knowledge']) for data in batch])
  max_len_response_sample = max([len(data['response']) for data in batch])
  
  padding_ind = 0 ## for bert is 0
  result_history = torch.zeros(len_batch, max_len_history)
  result_knowledge_sample = torch.zeros(len_batch, max_len_knowledge_sample)
  result_response_sample = torch.zeros(len_batch, max_len_response_sample)

  for i, data in enumerate(batch):
    p1 = len(data['history'])
    result_history[i, :p1] = data['history']
    p2 = len(data['knowledge'])
    result_knowledge_sample[i, :p2] = data['knowledge']
    p3 = len(data['response'])
    result_knowledge_sample[i, :p3] = data['response']



  return result_history.long(), result_knowledge_sample.long(), result_knowledge_sample.long()

sampler = torch.utils.data.SequentialSampler(dataset)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=128, sampler=sampler,
                                             shuffle=False, collate_fn=my_collate_fn)

In [22]:
print("Ok let's test!!!!")
print(distilbert_tokenizer.decode(dataset[10000]['history']))
print(distilbert_tokenizer.decode(dataset[10000]['knowledge']))
print(distilgpt_tokenizer.decode(dataset[10000]['response']))

Ok let's test!!!!
[CLS] i really enjoy japanese cousine, especially sushi. [SEP] me too, what type of rice do you have it with? [SEP]
[CLS] sushi can be prepared with either brown or white rice. [SEP]
[CLS] Most often white, basmati. But I use brown sometimes also. [SEP]


In [23]:
print("Ok let's test!!!!")
print((dataset[10000]['history']))
print((dataset[10000]['knowledge']))
print((dataset[10000]['response']))

Ok let's test!!!!
tensor([  101,  1045,  2428,  5959,  2887,  5542,  2063,  1010,  2926, 10514,
         6182,  1012,   102,  2033,  2205,  1010,  2054,  2828,  1997,  5785,
         2079,  2017,  2031,  2009,  2007,  1029,   102])
tensor([  101, 10514,  6182,  2064,  2022,  4810,  2007,  2593,  2829,  2030,
         2317,  5785,  1012,   102])
tensor([  58, 5097,   50,   60, 4042, 1690, 2330,   11, 1615, 6759,   72,   13,
         887,  314,  779, 7586, 3360,  635,   13,  685, 5188,   47,   60])


In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
for _, batch in enumerate(dataset_loader):
  history, knowledge, response = batch
  # [batch_size, len], [batch_size, len], [batch_size, len]

In [32]:
from transformers import AutoModel
model = AutoModel.from_pretrained('distilbert-base-uncased')
model.__dict__

{'_backward_hooks': OrderedDict(),
 '_buffers': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('wte', Embedding(50257, 768)),
              ('wpe', Embedding(1024, 768)),
              ('drop', Dropout(p=0.1, inplace=False)),
              ('h', ModuleList(
                 (0): Block(
                   (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                   (attn): Attention(
                     (c_attn): Conv1D()
                     (c_proj): Conv1D()
                     (attn_dropout): Dropout(p=0.1, inplace=False)
                     (resid_dropout): Dropout(p=0.1, inplace=False)
                   )
                   (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                   (mlp): MLP(
                     (c_fc): Conv1D()
                     (c_proj): Conv1D()
                     (dropout): Dropout(p=0.1, inplace=

In [0]:
import torch
from transformers import *
import numpy as np
import torch.nn as nn
from transformers import AutoModel


class Seq2SeqModel(nn.Module):
  
  def __init__(self, device):
    super().__init__()

    self.device = device
    
    self.dbert = AutoModel.from_pretrained('distilbert-base-uncased')

  def forward(self, history, knowledge):
    
    # history = [batch_size, history_len]
    # knowledge = [batch_size, knowledge_len]

    x = torch.cat((history, knowledge),1)
    temp = self.dbert(x)[0]

    return temp

In [0]:
model = Seq2SeqModel(device).to(device)

In [57]:
history_len = 20
batch_size = 64
vocab_size = 30000
test_history = torch.LongTensor(batch_size, history_len).random_(1,vocab_size).to(device)
knowledge_len = 20
test_knowledge = torch.LongTensor(batch_size, knowledge_len).random_(1,vocab_size).to(device)
model(test_history, test_knowledge).shape

torch.Size([64, 40, 768])